# CrewAI Agent with GitHub MCP Server

This notebook demonstrates how to build a **CrewAI** agent that connects to the
[GitHub MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/github)
using the Model Context Protocol (MCP). The agent uses a locally-hosted
**LLaMA 3.2** model via Ollama to reason and act on GitHub data.

---

## Architecture Overview

```
┌─────────────────────────────────────────────────────┐
│                      CrewAI Crew                    │
│                                                     │
│  ┌──────────────────────────────────────────────┐   │
│  │           GitHub Repository Analyst          │   │
│  │   LLM: Ollama / LLaMA 3.2 (lambda2.uncw.edu) │   │
│  │   Tools: GitHub MCP Server (via npx)         │   │
│  └──────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────┘
                          │
          MCP Stdio Transport (subprocess)
                          │
          @modelcontextprotocol/server-github
                          │
                   GitHub REST API
```

---

## Prerequisites

Run the installation cell below, then set your GitHub Personal Access Token
as an environment variable before proceeding.

## 1. Install Dependencies

Install the required Python packages and the GitHub MCP server (Node.js package).

> **Note:** The `npx` command below will auto-install `@modelcontextprotocol/server-github`
> on first use. You still need Node.js / npm available on your system.

In [ ]:
# Install Python dependencies
%pip install --quiet crewai crewai-tools mcp

## 2. Set GitHub Token

The GitHub MCP server needs a Personal Access Token (PAT) to authenticate with
the GitHub API. Create one at https://github.com/settings/tokens with at least
**repo** (read) scope.

Set it as an environment variable — **never hard-code tokens in notebooks**.

In [ ]:
import os

# Option A: already set in your shell environment
# export GITHUB_PERSONAL_ACCESS_TOKEN=ghp_...

# Option B: set it here (use getpass so the token isn't visible)
import getpass

if not os.environ.get("GITHUB_PERSONAL_ACCESS_TOKEN"):
    os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"] = getpass.getpass(
        "Enter your GitHub Personal Access Token: "
    )

github_token = os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"]
print("Token loaded:", "✅" if github_token else "❌ not set")

## 3. Import Libraries

| Import | Purpose |
|---|---|
| `crewai.LLM` | Wrapper around LLM backends (Ollama, OpenAI, etc.) |
| `crewai.Agent` | Defines a role-playing AI agent with goals and tools |
| `crewai.Task` | A unit of work assigned to an agent |
| `crewai.Crew` | Orchestrates agents and tasks |
| `crewai_tools.MCPServerAdapter` | Bridges MCP servers into CrewAI tools |
| `mcp.StdioServerParameters` | Config for launching an MCP server as a subprocess |

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import MCPServerAdapter
from mcp import StdioServerParameters

print("Imports successful ✅")

## 4. Configure the LLM

We use a locally-hosted **LLaMA 3.2** model served via
[Ollama](https://ollama.ai/) on `lambda2.uncw.edu`.

The `LLM` class from CrewAI accepts any LiteLLM-compatible model string, so
swapping to a different provider (OpenAI, Anthropic, etc.) is a one-line change.

In [ ]:
llm = LLM(
    model="ollama/llama3.2",
    base_url="http://lambda2.uncw.edu:11434/api/generate",
)

print(f"LLM configured: {llm.model}")

## 5. Configure the GitHub MCP Server

`StdioServerParameters` tells the `MCPServerAdapter` how to launch the MCP
server as a subprocess:

- **`command`** — the executable to run (`npx`)
- **`args`** — arguments passed to `npx`; `-y` auto-confirms the install
- **`env`** — environment variables forwarded to the subprocess (our token)

The adapter will start the process, perform the MCP handshake, and expose all
server-advertised tools as standard CrewAI tools.

In [ ]:
server_params = StdioServerParameters(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-github"],
    env={"GITHUB_PERSONAL_ACCESS_TOKEN": github_token},
)

print("MCP server parameters configured ✅")

## 6. Define the Agent

A CrewAI **Agent** has three key attributes that shape its behaviour:

| Attribute | Description |
|---|---|
| `role` | The agent's job title — used in prompts and logs |
| `goal` | What the agent is trying to achieve |
| `backstory` | Persona / context that influences reasoning style |

The agent is given the full list of GitHub MCP tools so it can call whichever
ones it needs to complete its task.

## 7. Define the Task

A **Task** is a concrete assignment given to an agent. Two fields matter most:

- **`description`** — natural-language instructions for what to do
- **`expected_output`** — tells the agent (and the Crew) what a successful
  result looks like, used for self-evaluation and hand-off

Feel free to change the description to explore any public GitHub repository!

## 8. Assemble and Run the Crew

Everything runs inside the `MCPServerAdapter` context manager, which:

1. Spawns the `npx` subprocess
2. Performs the MCP protocol handshake
3. Yields the list of available tools
4. Cleans up the subprocess on exit

`crew.kickoff()` starts the agentic loop — the agent will reason, call GitHub
tools, observe results, and iterate until it produces a final answer.

In [ ]:
with MCPServerAdapter(server_params) as github_tools:

    # ── Inspect available tools ────────────────────────────────────────────
    print("Available GitHub MCP tools:")
    for tool in github_tools:
        print(f"  • {tool.name}")

    # ── Agent ──────────────────────────────────────────────────────────────
    github_agent = Agent(
        role="GitHub Repository Analyst",
        goal=(
            "Retrieve and analyse information from GitHub repositories "
            "to answer the user's question accurately."
        ),
        backstory=(
            "You are an expert software engineer who specialises in "
            "exploring GitHub repositories, reading issues, pull requests, "
            "and code to provide clear, concise summaries and insights."
        ),
        tools=github_tools,
        llm=llm,
        verbose=True,
    )

    # ── Task ───────────────────────────────────────────────────────────────
    research_task = Task(
        description=(
            "Search GitHub for the repository 'anthropics/anthropic-sdk-python'. "
            "Retrieve the repository details (description, stars, open issues) "
            "and list the 5 most recent open issues with their titles and URLs."
        ),
        expected_output=(
            "A structured summary containing:\n"
            "1. Repository description and key stats (stars, forks, open issues).\n"
            "2. A numbered list of the 5 most recent open issues with title and URL."
        ),
        agent=github_agent,
    )

    # ── Crew ───────────────────────────────────────────────────────────────
    crew = Crew(
        agents=[github_agent],
        tasks=[research_task],
        verbose=True,
    )

    # ── Run ────────────────────────────────────────────────────────────────
    result = crew.kickoff()

## 9. Display the Final Result

In [ ]:
print("=" * 60)
print("CREW RESULT")
print("=" * 60)
print(result)

---

## Next Steps

- **Change the task** — update the `description` in the Task cell to query any
  other public repository or ask a different question.
- **Add more agents** — create specialist agents (e.g. a Code Reviewer, a
  Release Note Writer) and wire them together in the Crew.
- **Swap the LLM** — replace `ollama/llama3.2` with `openai/gpt-4o` or any
  other LiteLLM-supported model string.
- **Explore other MCP servers** — the same pattern works with any MCP-compatible
  server (filesystem, Slack, databases, etc.).